# Intro

 - **Mục tiêu**: Mong muốn sinh 1 mẫu với phân phối f nào đó.  
 - **Phương pháp sử dụng**: Sinh Markov Chain $\rightarrow$ có phân phối dừng là hàm $f$ mong muốn.  

## Vấn đề tích phân trong thống kê Bayesian

Giả sử có 1 mẫu ${x_1, x_2, ..., x_n}$ ta  
Định nghĩa ra 1 hàm likelihood $f(x_1, x_2, ..., x_n | \theta)$

Sử dụng công thức Bayes ta có thể tính được $f(\theta | x)$  
Từ đó tính được kỳ vọng $E[g(\theta | x)]$ hay chính là $E[g(Y)]$

## Tích phân Markov Chain Monte Carlo  

Tính chất của chuỗi Markov 

# Phương pháp con 1. Thuật toán Metropolis-Hastings 

 - **Mục tiêu**: Sinh ra 1 chuỗi Markov Chain  
 - **Cách thức**: Việc sinh mẫu. Giả thiết có 1 điểm data nào đó. Dựa trên phân phối đề xuất $g(. | X_t)$ nào đó $\rightarrow$ Chấp nhận OR Bác bỏ điểm được sinh ra tiếp theo dựa trên phân phối $g$. Như vậy có thể gọi là sig Markov (cái tạo sau liên quan tới cái tạo trước đó)

Điều đó đồng nghĩa $g(.|X_t)$ sẽ thay đổi liên tục tùy thuộc vào $X_t$ đang xét hiện tại.

Vấn đề là chọn phân phối đề xuất $g$ như thế nào đi nữa (phải đảm bảo điều kiện **giống họ các phân phối - class distribution**) thì sẽ đảm bảo hội tụ.  
Tuy nhiên nếu phân phối $g$ được chọn là đủ tốt thì sẽ giúp tốc độ hội tụ (không bác bỏ liên tục) sẽ nhanh hơn. Từ đó giúp việc sinh ra mẫu $X$ nhanh, tránh mất thời gian hơn.

## Thuật toán

Đầu tiên ta tạo $X_0$ từ phân phối $g$ đã chọn. Bây giờ liên tục lặp cho đến khi tạo đủ số điểm data cho mẫu mong muốn: 

 - B1. Tạo $Y$ theo phân phối $g(.|X_t)$  
 - B2. Tạo $U$ từ phân phối Chuẩn (0, 1)  
 - B3. Nếu điều kiện $U \leq \frac{f(Y)g(X_t|Y)}{f(X_t)g(Y|X_t)}$ được thỏa mãn thì ta chấp nhận điểm $Y$ được tạo ra ($X_{t+1} = Y$) còn không thì $X_{t+1} = X_{t}$.

In [1]:
f <- function(x, sigma) {
    if (any(x < 0)) {
        return(0)
    }
    stopifnot(sigma > 0)
    return((x / sigma^2) * exp(-x^2 / (2 * sigma^2)))
}

m <- 10000
sigma <- 4
x <- numeric(m)
x[1] <- rchisq(1, df = 1)
k <- 0
u <- runif(m)
for (i in 2:m) {
    xt <- x[i - 1]
    y <- rchisq(1, df = xt)
    num <- f(y, sigma) * dchisq(xt, df = y) # g là hàm dchisq
    den <- f(xt, sigma) * dchisq(y, df = xt)
    if (u[i] <= num / den) {
        x[i] <- y
    } else {
        x[i] <- xt
        k <- k + 1
        # y is rejected
    }
}
print(k)

[1] 4073


Thuật toán Metropolis-Hastings là một phương pháp quan trọng trong Markov Chain Monte Carlo (MCMC) được sử dụng để lấy mẫu từ một phân phối có phân phối dư luận phức tạp. Trong thuật toán này, chúng ta sử dụng một quá trình Markov để di chuyển qua các trạng thái khác nhau của không gian mẫu, và chúng ta chấp nhận hoặc từ chối mỗi bước của quá trình Markov này dựa trên một tỷ lệ được gọi là tỷ lệ chấp nhận.

Trong đoạn mã bạn đã cung cấp, chúng ta sử dụng thuật toán Metropolis-Hastings để lấy mẫu từ một phân phối chisquare bằng cách sử dụng một hàm mật độ xác suất không chuẩn. Dưới đây là ý nghĩa của mỗi phần của đoạn mã:

1. Hàm f(x, sigma): Đây là hàm mật độ xác suất không chuẩn được sử dụng trong phương pháp Metropolis-Hastings để xác định tỷ lệ chấp nhận. Trong trường hợp này, hàm này được định nghĩa để trả về giá trị 0 nếu bất kỳ giá trị x nào nhỏ hơn 0, và trả về một giá trị phức tạp dựa trên x nếu x lớn hơn hoặc bằng 0.

2. Mẫu ban đầu và số lần từ chối (k): Ban đầu, chúng ta chọn một giá trị ngẫu nhiên cho x từ phân phối chisquare với một độ tự do. Trong vòng lặp, chúng ta tính toán một số lượng mẫu m mới từ phân phối chisquare với một độ tự do dựa trên giá trị x hiện tại, sau đó xác định liệu chúng ta nên chấp nhận mẫu mới này hay không dựa trên tỷ lệ chấp nhận.

3. Vòng lặp Metropolis-Hastings: Trong mỗi vòng lặp, chúng ta lấy mẫu một giá trị y từ phân phối chisquare với một độ tự do, sau đó tính tỷ lệ chấp nhận num/den dựa trên hàm f và hàm mật độ xác suất chisquare. Nếu giá trị của u (một biến ngẫu nhiên từ phân phối đồng nhất) nhỏ hơn tỷ lệ chấp nhận, chúng ta chấp nhận mẫu mới y; nếu không, chúng ta tiếp tục với giá trị hiện tại của x và tăng biến k để đếm số lần từ chối.

4. Kết quả: Cuối cùng, chúng ta in ra số lần từ chối k, mà có thể là một chỉ số về sự hiệu quả của phương pháp Metropolis-Hastings. Đối với một phân phối phức tạp, số lần từ chối có thể cao, nhưng chúng ta vẫn có thể thu thập một chuỗi mẫu có phân phối gần với phân phối mục tiêu.